In [5]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [6]:
# batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
# num_classes = 5 # 類別的數量，共有 5 個類別
# epochs = 10 # 訓練的 epochs 數量

# # 讀取資料並檢視
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # 對 label 進行 one-hot encoding (y_trian 原本是純數字)
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)


DATASET_PATH  = './image_data/train'
TEST_PATH = './image_data/test'
OUTPUT_PATH = './working'
NUM_CLASSES = 5
NUM_EPOCHS = 10
SEED = 77
# saved model
WEIGHTS_FINAL = 'model-InceptionResNetV2.h5'
categories=os.listdir(DATASET_PATH)



In [7]:
train_datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.125, height_shift_range=0.125, zoom_range=0.125, horizontal_flip=True,
                                   validation_split=0.2, rescale=1. / 255)
train_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'training', seed = SEED)
valid_batches = train_datagen.flow_from_directory(DATASET_PATH, subset = 'validation', seed = SEED)
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.
Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


In [8]:
# Build Model
net = InceptionResNetV2(include_top=False, input_shape=(256, 256, 3))
x = net.output
x = Flatten()(x)
# Add Dense layer, each probability by softmax
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
model = Model(inputs=net.input, outputs=output_layer)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# Train
model.fit_generator(train_batches, validation_data = valid_batches, epochs = NUM_EPOCHS)

# Store Model
model.save(WEIGHTS_FINAL)

Epoch 1/10
71/71 [==============================] - 1418s 20s/step - loss: 2.6434 - accuracy: 0.5406 - val_loss: 799468.1875 - val_accuracy: 0.1705
Epoch 2/10
71/71 [==============================] - 1379s 19s/step - loss: 2.7457 - accuracy: 0.6801 - val_loss: 2486951.5000 - val_accuracy: 0.1936
Epoch 3/10
71/71 [==============================] - 1368s 19s/step - loss: 2.5009 - accuracy: 0.6900 - val_loss: 2433.8774 - val_accuracy: 0.2558
Epoch 4/10
71/71 [==============================] - 1417s 20s/step - loss: 2.0164 - accuracy: 0.7067 - val_loss: 151.5958 - val_accuracy: 0.2682
Epoch 5/10
71/71 [==============================] - 1728s 24s/step - loss: 1.2451 - accuracy: 0.7660 - val_loss: 10624.8809 - val_accuracy: 0.2345
Epoch 6/10
71/71 [==============================] - 1934s 27s/step - loss: 0.8818 - accuracy: 0.7882 - val_loss: 929.5561 - val_accuracy: 0.3037
Epoch 7/10
71/71 [==============================] - 1412s 20s/step - loss: 0.8431 - accuracy: 0.7969 - val_loss: 3.9767 

In [9]:
from keras.preprocessing import image
out = np.array(['id', 'flower_class'])
testfiles=os.listdir(TEST_PATH)
print(OUTPUT_PATH+'/submission.csv')
for testfile in testfiles:
    filename = testfile.split('.')[0]
    img = image.load_img(TEST_PATH+'/'+testfile,target_size=(256, 256))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = x /255.
    pred = model.predict(x)[0]
    tof=np.argmax(pred)
    out = np.vstack((out,[filename, tof]))

pd.DataFrame(out).to_csv(OUTPUT_PATH+'/submission.csv',index=False,header=False)

./working/submission.csv
